# Analyzing and Clustering Toronto Neighborhoods

In [25]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import folium
import requests
import geocoder
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html

# 1. WebScrapping Data

In [26]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urlopen(url)
html = page.read().decode('UTF-8')
soup = BeautifulSoup(html, 'html.parser')

In [27]:
#print(soup.prettify()) #prints the parsed data of html

In [28]:
table = soup.find('table', attrs = {"class":"wikitable"})
table_data = table.tbody.find_all('tr')

TableHeadings = []
for i in range(3):
    th = table_data[0].find_all('th')
    TableHeadings.append(th[i].text.replace('\n',' ').strip())
print(TableHeadings)

['Postal Code', 'Borough', 'Neighbourhood']


# 2. Clean and Preparing Data 

In [29]:
Postal_data = []
Borough_data = []
Neigh_data = []
for i in range(len(table_data)-1):
    td = table_data[i+1].find_all('td')
    for j in range(1): 
        if td[1].text.replace('\n', ' ').strip() == 'Not assigned':
            continue
        elif td[2].text.replace('\n', ' ').strip() == 'Not assigned':
            Postal_data.append(td[0].text.replace('\n', ' ').strip())
            Borough_data.append(td[1].text.replace('\n', ' ').strip())
            Neigh_data.append(td[1].text.replace('\n', ' ').strip())
        else:
            Postal_data.append(td[0].text.replace('\n', ' ').strip())
            Borough_data.append(td[1].text.replace('\n', ' ').strip())
            Neigh_data.append(td[2].text.replace('\n', ' ').strip())

In [30]:
Neigh_Toronto_data = [Postal_data, Borough_data, Neigh_data]
df1 = pd.DataFrame(data = Neigh_Toronto_data).T #Transposing dataframe
df1.columns = TableHeadings #Giving the dataframe column names
df1

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [31]:
df1.shape

(103, 3)

# 3. Obtaining the Geographical coordinates 

In [9]:
Latitude=[]
Longitude=[]

for code in df1['Postal Code']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    #print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        #print(code, g.latlng)
    latlng = g.latlng
    Latitude.append(latlng[0])
    Longitude.append(latlng[1])

In [10]:
Latlng_Toro_data = [Postal_data, Borough_data, Neigh_data, Latitude, Longitude]
Labels = ['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
df2 = pd.DataFrame(data= Latlng_Toro_data).T
df2.columns = Labels
df2

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7525,-79.3299
1,M4A,North York,Victoria Village,43.7306,-79.3131
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6551,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7233,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6625,-79.3919
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6532,-79.5111
99,M4Y,Downtown Toronto,Church and Wellesley,43.6666,-79.3813
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.6487,-79.3854
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6328,-79.4894


<b>Analysis</b> Working with data containing word 'Toronto'

In [11]:
Toronto_data = df2[df2['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6551,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6625,-79.3919
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6574,-79.378
3,M5C,Downtown Toronto,St. James Town,43.6522,-79.3759
4,M4E,East Toronto,The Beaches,43.6771,-79.2955
5,M5E,Downtown Toronto,Berczy Park,43.6454,-79.3731
6,M5G,Downtown Toronto,Central Bay Street,43.6561,-79.3849
7,M6G,Downtown Toronto,Christie,43.6687,-79.4207
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6497,-79.3826
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.6651,-79.4389


In [12]:
from geopy.geocoders import Nominatim 
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Visualizing the Toronto Neighborhood 

In [32]:
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

# 4. Get Nearby Venues in Toronto

###### Applying FourSquare API 

In [39]:
CLIENT_ID = '3PHPL1LNPWCN4TKCCFJRJC4IYCAULMBAT1TYWW312F3PFZTH' # your Foursquare ID
CLIENT_SECRET ='V3Q32WCU3KLJJAYWPYRUUCIUEKMKWLHRQKZRKCDBLUT5QKR0'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3PHPL1LNPWCN4TKCCFJRJC4IYCAULMBAT1TYWW312F3PFZTH
CLIENT_SECRET:V3Q32WCU3KLJJAYWPYRUUCIUEKMKWLHRQKZRKCDBLUT5QKR0


In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
Toronto_venues = getNearbyVenues(Toronto_data['Neighborhood'], Toronto_data['Latitude'], Toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [42]:
Toronto_venues.shape

(1752, 7)

In [43]:
Toronto_venues.head(5)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


# 5. Explore the neighborhoods

In [44]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,60,60,60,60,60,60
"Brockton, Parkdale Village, Exhibition Place",87,87,87,87,87,87
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",76,76,76,76,76,76
Central Bay Street,76,76,76,76,76,76
Christie,11,11,11,11,11,11
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,27,27,27,27,27,27


In [45]:
Toronto_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,1,1,1,1,1,1
Afghan Restaurant,1,1,1,1,1,1
American Restaurant,24,24,24,24,24,24
Antique Shop,2,2,2,2,2,2
Aquarium,2,2,2,2,2,2
...,...,...,...,...,...,...
Wine Bar,11,11,11,11,11,11
Wine Shop,1,1,1,1,1,1
Wings Joint,1,1,1,1,1,1


In [46]:
print('The number of unique categories are {}'.format(len(Toronto_venues['Venue Category'].unique())))

The number of unique categories are 232


# 6. Analyzing Each Neighborhood 

In [47]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [48]:
#One hot encoding 
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']],prefix="",prefix_sep="")

#Add neighborhood colum back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues["Neighborhood"]

In [49]:
Toronto_onehot.shape

(1752, 232)

#### Grouping rows by neighborhood and by taking the mean of frequency of occurence of each category 

In [50]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [51]:
Toronto_grouped.shape

(38, 232)

#### Printing each neighborhood along with the top 5 most common venues

In [52]:
num_top_venues = 5 
for hood in Toronto_grouped['Neighborhood']:
    print("-----"+hood+"-----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood']==hood].T.reset_index()
    temp.columns = ['Venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print("\n")

-----Berczy Park-----
                Venue  freq
0         Coffee Shop  0.08
1         Cheese Shop  0.03
2          Restaurant  0.03
3  Seafood Restaurant  0.03
4              Bakery  0.03


-----Brockton, Parkdale Village, Exhibition Place-----
            Venue  freq
0     Coffee Shop  0.06
1            Café  0.06
2             Bar  0.06
3      Restaurant  0.05
4  Sandwich Place  0.03


-----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto-----
                Venue  freq
0         Coffee Shop  0.10
1               Hotel  0.05
2          Restaurant  0.04
3                Café  0.03
4  Italian Restaurant  0.03


-----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport-----
                Venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.07
2                Café  0.07
3   French Restaurant  0.04
4                 Bar  0.04


-----Central Bay Street-----
              Venue  freq


In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return(row_categories_sorted.index.values[0:num_top_venues])

In [54]:
num_top_venues = 10 

s = ['st', 'nd', 'rd']

#Create columns according to number of top veneus 
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped["Neighborhood"]

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:] = return_most_common_venues(Toronto_grouped.iloc[ind,:],num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Beer Bar,Farmers Market,Seafood Restaurant,Cheese Shop,Breakfast Spot,Restaurant,Cocktail Bar,Bakery,Museum
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bar,Coffee Shop,Restaurant,Sandwich Place,Nightclub,Gift Shop,Lounge,Japanese Restaurant,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Restaurant,Café,Asian Restaurant,Italian Restaurant,Bar,Sushi Restaurant,Mediterranean Restaurant,Seafood Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Café,Bar,Park,French Restaurant,Gym / Fitness Center,Speakeasy,Bakery,Sandwich Place
4,Central Bay Street,Coffee Shop,Clothing Store,Hotel,Bubble Tea Shop,Middle Eastern Restaurant,Sandwich Place,Café,Restaurant,Sushi Restaurant,Plaza


# 6. Clustering Each Neighborhood

Run k-means to cluster the neighborhood into 5 clusters

In [55]:
from sklearn.cluster import KMeans

#setting the number of clusters 
kclusters = 5
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [56]:
#Dropping the Clustering labels
try:
    neighborhoods_venues_sorted.drop('Cluster Labels', 1, inplace=True)
    neighborhoods_venues_sorted.insert(0,'Cluster Labels',kmeans.labels_)
except:
    #adding Clustering Labels
    neighborhoods_venues_sorted.insert(0,'Cluster Labels',kmeans.labels_)

In [57]:
Toronto_merged = Toronto_data
#Merge Toronto_grouped with Toronto_data to add latitude&Latitude
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')

In [58]:
#Removing a row of Nan data
Toronto_merged.drop(Toronto_merged.index[19],inplace=True)
Toronto_merged.reset_index(drop = True, inplace = True)

###### Visualizing the resulting clusters

In [59]:
import folium
import matplotlib.cm as cm 
import matplotlib.colors as colors

In [60]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)

#set color scheme for the clusters 
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers to the map 
markers_colors = []
for lat, lon, neigh, cluster in zip(Toronto_merged["Latitude"],Toronto_merged['Longitude'],Toronto_merged["Neighborhood"], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(neigh) + ' Cluster ' + str(cluster),parse_html = True)
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius = 5, 
        popup=label, 
        color=rainbow[cluster-1], 
        fill = True, 
        fill_color = rainbow[cluster-1], 
        fill_opacity = 0.7).add_to(map_clusters)
    
map_clusters

# 7. Analyzing Each Neighborhood Cluster

<b>Cluster 1</b>

In [61]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1]+ list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0.0,Coffee Shop,Breakfast Spot,Yoga Studio,Italian Restaurant,Spa,Event Space,Food Truck,Electronics Store,Bakery,Thai Restaurant
1,Downtown Toronto,0.0,Coffee Shop,Sandwich Place,Mediterranean Restaurant,Italian Restaurant,Café,Falafel Restaurant,Fried Chicken Joint,Bank,Theater,Gastropub
2,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Furniture / Home Store,Theater,Ramen Restaurant,Bookstore,Movie Theater
3,Downtown Toronto,0.0,Coffee Shop,Cocktail Bar,Restaurant,Clothing Store,Gastropub,Café,Hotel,Cosmetics Shop,Beer Bar,Lingerie Store
5,Downtown Toronto,0.0,Coffee Shop,Beer Bar,Farmers Market,Seafood Restaurant,Cheese Shop,Breakfast Spot,Restaurant,Cocktail Bar,Bakery,Museum
6,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Hotel,Bubble Tea Shop,Middle Eastern Restaurant,Sandwich Place,Café,Restaurant,Sushi Restaurant,Plaza
7,Downtown Toronto,0.0,Café,Grocery Store,Playground,Baby Store,Italian Restaurant,Athletics & Sports,Coffee Shop,Candy Store,Farmers Market,Farm
8,Downtown Toronto,0.0,Hotel,Coffee Shop,Café,Japanese Restaurant,Restaurant,Gym,Asian Restaurant,American Restaurant,Steakhouse,Salad Place
9,West Toronto,0.0,Park,Grocery Store,Convenience Store,Smoke Shop,Skating Rink,Middle Eastern Restaurant,Brazilian Restaurant,Furniture / Home Store,Music Venue,Bar
10,Downtown Toronto,0.0,Coffee Shop,Hotel,Japanese Restaurant,Restaurant,Plaza,Deli / Bodega,Aquarium,Boat or Ferry,Park,Music Venue


<b> Cluster 2 </b>

In [62]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1]+ list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,1.0,Bus Line,Swim School,Yoga Studio,Food,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


<b> Cluster 3 </b> 

In [63]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1]+ list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Central Toronto,2.0,French Restaurant,Park,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
22,Central Toronto,2.0,Playground,Gym Pool,Park,Yoga Studio,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


<b> Cluster 4 </b> 

In [64]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1]+ list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,3.0,Health Food Store,Pub,Trail,Yoga Studio,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


<b> Cluster 5</b>

In [65]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1]+ list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,4.0,Playground,Gym,Trail,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
